In [1]:
!pip install tokenizers --force

distutils: /u/kem5en/.conda/envs/ml_proj/include/python3.6m/UNKNOWN
sysconfig: /u/kem5en/.conda/envs/ml_proj/include/python3.6m
user = False
home = None
root = None
prefix = None
  Using cached tokenizers-0.10.2-cp36-cp36m-manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.2
    Uninstalling tokenizers-0.10.2:
      Successfully uninstalled tokenizers-0.10.2
distutils: /u/kem5en/.conda/envs/ml_proj/include/python3.6m/UNKNOWN
sysconfig: /u/kem5en/.conda/envs/ml_proj/include/python3.6m
user = False
home = None
root = None
prefix = None


In [2]:
!pip install -U ray

distutils: /u/kem5en/.conda/envs/ml_proj/include/python3.6m/UNKNOWN
sysconfig: /u/kem5en/.conda/envs/ml_proj/include/python3.6m
user = False
home = None
root = None
prefix = None
distutils: /u/kem5en/.conda/envs/ml_proj/include/python3.6m/UNKNOWN
sysconfig: /u/kem5en/.conda/envs/ml_proj/include/python3.6m
user = False
home = None
root = None
prefix = None


In [3]:
# %%
#!pip install pytorch-lightning transformers keras matplotlib numpy sklearn nltk pytorch-pretrained-bert pytorch-nlp matplotlib sklearn keras
#Lightning Imports
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from ray.tune.integration.pytorch_lightning import TuneReportCallback
import ray.tune

#Other Imports
import pandas as pd
import re
from torch.utils.data import TensorDataset, RandomSampler, DataLoader, SequentialSampler
import numpy as np
from transformers import BertForSequenceClassification
#from transformers import BerttokenizerFast
from transformers import AutoTokenizer
from pytorch_lightning.loggers import TensorBoardLogger
import pickle

#from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

# from argparse import ArgumentParser

class ProjData(pl.LightningDataModule):

    def __init__(self, data_dir: str = "~/tmp", batch_size: int = 32, max_len : int = 128, ratio : int = 2):
        super().__init__()
        self.data_dir = data_dir
        # self.batch_size = batch_size
        self.max_len = max_len # Bert Max Len input
        self.rat = ratio
        self.name = "a-" + str(max_len) + "-" + str(ratio)
        self.tokenizer = None

    def setup(self, stage=None):
        
        # *** tokenizer isn't actually a constant and the do_lower_case should be redundant if preprocessing was correct.
        # self.tokenizer = BerttokenizerFast.from_pretrained(MODEL_NAME, do_lower_case=False)
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', use_fast=True, do_lower_case=True)


        ###
        # Preprocessing Data
        ###
        print("Getting pos")
        pos_data = self.get_pos_dataset()
        print("Getting neg")
        neg_data = self.get_neg_dataset()

        # *** TODO: Proper subset selection either in concat_datasets or in get_neg_dataset
        print("Joining")
        dataset = self.concat_datasets(pos_data, neg_data)
        dataset.dropna(inplace=True) # Losing negative examples potentially...
        dataset["data"] = dataset["data"].map(self.preprocess)

        # 60% - train set, 20% - validation set, 20% - test set
        train, validate, test = np.split(dataset.sample(frac=1, random_state=42), 
                       [int(.6*len(dataset)), int(.8*len(dataset))])

        X_train, y_train = train["data"], train["label"]
        X_val, y_val = validate["data"], validate["label"]
        X_test, y_test = test["data"], test["label"]    


        

        # NOTE: This is a small subset used for testing... likely will remove in final ver.
        # *** Set to "None" to skip - if there any reason you take the tails of the lists rather than the heads?
        PROTOTYPE_NUM = None
        # **** NEED TO SHUFFLE FIRST IF YOU USE THIS
        if PROTOTYPE_NUM:
            X_train = X_train[:PROTOTYPE_NUM * 10]
            y_train = y_train[:PROTOTYPE_NUM * 10]
            X_val = X_val[:PROTOTYPE_NUM]
            y_val = y_val[:PROTOTYPE_NUM]
            X_test = X_test[:PROTOTYPE_NUM]
            y_test = y_test[:PROTOTYPE_NUM]

        ###
        # Tokenization
        ###
        # Convert texts into tokens. (These are not truncated or padded yet)
        print("Tokenizing")
        pre_train_input_ids = self.tokenize_datasets(X_train, self.tokenizer)
        pre_val_input_ids = self.tokenize_datasets(X_val, self.tokenizer)
        pre_test_input_ids = self.tokenize_datasets(X_test, self.tokenizer)
        
        # Truncate and Pad your tokens
        print("Padding")
        train_input_ids = self.trunc_n_pad(pre_train_input_ids)
        val_input_ids = self.trunc_n_pad(pre_val_input_ids)
        test_input_ids = self.trunc_n_pad(pre_test_input_ids)

        ###
        # Misc.
        ###
        # Create attention masks
        print("Creating masks")
        train_attention_masks = self.create_attention_masks(train_input_ids)
        val_attention_masks = self.create_attention_masks(val_input_ids)
        test_attention_masks = self.create_attention_masks(test_input_ids)

        # # Convert all of our data into torch tensors, the required datatype for our model
        train_inputs = torch.tensor(train_input_ids)
        validation_inputs = torch.tensor(val_input_ids)

        train_labels = torch.tensor(y_train.values.tolist())
        validation_labels = torch.tensor(y_val.values.tolist())

        train_masks = torch.tensor(train_attention_masks)
        validation_masks = torch.tensor(val_attention_masks)

        test_inputs = torch.tensor(test_input_ids)
        test_labels = torch.tensor(y_test.values.tolist())

        test_masks = torch.tensor(test_attention_masks)

        # Create an iterator of our data with torch DataLoader. 
        self.train = TensorDataset(train_inputs, train_masks, train_labels)
        self.val = TensorDataset(validation_inputs, validation_masks, validation_labels)
        self.test = TensorDataset(test_inputs, test_masks, test_labels)
        print(train_inputs)
        print(self.train)

    def process(self, sent):
        s = self.preprocess(sent)
        s = self.tokenize_datasets([s], self.tokenizer)
        s = self.trunc_n_pad(s)
        mask = self.create_attention_masks(s)
        return  torch.tensor(s),  torch.tensor(mask)




    def preprocess(self, x):

        # self.re1 = re.compile('\n') 
        # self.re2 = re.compile('[\w]+')
        x = re.sub(r'(\n+)',r' ', x)
        # lower case in tokenizer
        x = " ".join(re.findall('[\w]+',x))
        return x
        
        # # RNV uses a special preprocess step --- mirror it for pushio.
        # # print("Preprocessing... 1. split new lines, 2. convert to lowercase, and 3. strip numbers and punct")
        # ### 1) remove newlines
        # data['data'] = data['data'].replace('\n', ' ', regex = True)

        # ## 2) convert to lowercase
        # data['data'] = data['data'].str.lower()

        # # ### 3) remove punct and numbers: https://stackoverflow.com/questions/47947438/preprocessing-string-data-in-pandas-dataframe
        # data["data"] = data.data.apply(lambda x : " ".join(re.findall('[\w]+',x)))
 

    # Import PushIO CSV    
    def get_neg_dataset(self, path="/bigtemp/rm5tx/nlp_project/2016-05_all.csv"):
        # path = "/localtmp/rm5tx/nlp_project/2016-05_all.csv"
        print("reading neg")
        data = pd.read_csv(path, usecols=['body'], dtype="string")
        data.rename(columns={"body":"data"}, inplace=True)
        # print("filtering n/a neg")
        # data.dropna(inplace=True)
        # print("preprocessing neg") --- less work to do this on the joined
        # data["data"] = data["data"].map(self.preprocess)
        data["label"] = 0
        # We want a unify col name for when we concat pos and neg data

        return data

    # Reddit Norm Violations
    def get_pos_dataset(self, path="/bigtemp/rm5tx/nlp_project/reddit-norm-violations/data/macro-norm-violations/"):
        directory = os.path.abspath(path)

        pos_temp = []
        for root, dirs, files in os.walk(directory):
            for file in files:
                with open(root+ "/" +file) as f:
                    pos_temp += f.readlines()
        data = pd.DataFrame(data=pos_temp, dtype = "string")
        data.rename(columns={0:"data"}, inplace=True)
        data["label"] = 1
        return data

    def get_adjacents_datasets(self):
        neg_init = self.get_neg_dataset()
        pos_init = self.get_neg_dataset()
        pos, neg = 0, 0
        return pos, neg

    def concat_datasets(self, data_a, data_b):
        if len(data_a.index) > len(data_b.index):
            data_a, data_b = data_b, data_a
        # frames = [data_a, data_b[(self.rat * len(data_a.index)):]]
        trunced_b = data_b.sample(n=int(self.rat * len(data_a.index)), random_state=42)
        print("Using ", len(data_a.index), len(trunced_b.index), " samples.")
        dataset = pd.concat([data_a, trunced_b])
        print("Total = ", len(dataset.index))
        return dataset

    ###Pre-processing Code###
    def tokenize_datasets(self, X_dataset, tokenizer):
        input_ids = []
        for sent in X_dataset:
            tokenized_text = tokenizer.encode(
                                            sent,                      # Sentence to encode
                                            add_special_tokens = True, # Add '[CLS]' and '[SEP]' tokens
                                            max_length = self.max_len,      # Truncate senences
                                            truncation=True,
                                            )
            input_ids.append(tokenized_text)
        return input_ids


    # Appears that CS Serv don[t have tf version 2.2]
    # Thus, we cannot use the convenient pad_sequences from keras
    def trunc_n_pad(self, input_id_list):
        ret_list = []
        for input_id in input_id_list:
            if len(input_id) > self.max_len:
                ret_list.append(input_id[:self.max_len])
            elif len(input_id) < self.max_len:
                temp_sublist = input_id + [0] * (self.max_len - len(input_id))
                ret_list.append(temp_sublist)
            else:
                ret_list.append(input_id)
        return ret_list

    # Create attention masks
    def create_attention_masks(self, input_ids):
        attention_masks = []
        for seq in input_ids:
            # Create a mask of 1s for each token followed by 0s for padding
            seq_mask = [float(i>0) for i in seq]
            attention_masks.append(seq_mask)
        return attention_masks

    def train_dataloader(self, batch_size=32):
        return DataLoader(self.train, batch_size=batch_size, shuffle=True)

    def val_dataloader(self, batch_size=32):
        return DataLoader(self.val, batch_size=batch_size)

    def test_dataloader(self, batch_size=32):
        return DataLoader(self.test, batch_size=sbatch_size)

    def save(self, path="/bigtemp/rm5tx/nlp_project/data_cache/"):
        path = "/bigtemp/jw6qs/"

        torch.save(self.tokenizer, open(path+self.name+"token.p", "wb"))
        torch.save(self.train, open(path+self.name+"train.pt", "wb"))
        torch.save(self.val, open(path+self.name+"val.pt", "wb"))
        torch.save(self.test, open(path+self.name+"test.pt", "wb"))

    def load(self, path="/bigtemp/rm5tx/nlp_project/data_cache/"):
        path = "/bigtemp/jw6qs/"

        self.tokenizer = torch.load(open(path+self.name+"token.p", "rb"))
        self.train = torch.load(open(path+self.name+"train.pt", "rb"))
        self.val = torch.load(open(path+self.name+"val.pt", "rb"))
        self.test = torch.load(open(path+self.name+"test.pt", "rb"))    

class ProjModel(pl.LightningModule):
    def __init__(
        self,
        # model_name_or_path: str,
        num_labels: int = 2,
        learning_rate: float = 2e-5,
        adam_epsilon: float = 1e-9,
        warmup_steps: int = 0,
        weight_decay: float = 0.0,
        # eval_splits: Optional[list] = None,
        **kwargs
        
    ):
        super().__init__()

        self.save_hyperparameters()
        self.model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
        # self.config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)
        # self.metric = ...

    def forward(self, x, mask):
        logits = self.model(x, attention_mask=mask).logits
        # return logits        
        pred = torch.argmax(logits, 1)
        return pred

    def training_step(self, batch, batch_idx):
        b_input_ids, b_input_mask, b_labels = batch
        out = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        logits = out.logits
        loss = out.loss
        acc = self.accurate_nb(logits, b_labels)

        self.log('train_acc', acc)
        self.log('train_loss', loss)
        return loss

    # *** Old Monitor, NYI -- Do we want to use a scheduler at all?
    # https://pytorch-lightning.readthedocs.io/en/latest/common/optimizers.html#automatic-optimization
    #     # for batch in validation_dataloader:
    #     #     with torch.no_grad():
    #     #         logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0] 

    #     #     tmp_eval_nb = accurate_nb(logits, b_labels)

    #     #     eval_accurate_nb += tmp_eval_nb
    #     #     nb_eval_examples += label_ids.shape[0]
    #     # eval_accuracy = eval_accurate_nb/nb_eval_examples
    #     # print("Validation Accuracy: {}".format(eval_accuracy))
    #     # scheduler.step(eval_accuracy)


    def validation_step(self, batch, batch_idx):
        b_input_ids, b_input_mask, b_labels = batch
        loss = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels).loss
        logits = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask).logits
        
        acc = self.accurate_nb(logits, b_labels)

        self.log('valid_acc', acc)
        self.log('valid_loss', loss, on_step=True)
        return loss


    def test_step(self, batch, batch_idx):
        b_input_ids, b_input_mask, b_labels = batch
        loss = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels).loss
        logits = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask).logits

        acc = self.accurate_nb(logits, b_labels)

        self.log('test_acc', acc)
        self.log('test_loss', loss)
        return loss

    def configure_optimizers(self):
        ###
        # Param Optim.
        ###
        param_optimizer = list(self.named_parameters())
        no_decay = ['bias', 'gamma', 'beta']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
            'weight_decay_rate': self.hparams.weight_decay},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
            'weight_decay_rate': 0.0}
        ]
        optimizer = torch.optim.Adam(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.1)
        # return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'eval_acc'}
        return optimizer
    

    def accurate_nb(self, logits, labels):
        pred_flat = torch.argmax(logits, dim=1).flatten()
        labels_flat = labels.flatten()
        return torch.sum(pred_flat == labels_flat) / labels_flat.shape[0]
    


In [4]:
def main():
    TRAIN_BATCH_SIZE = 32
    VAL_BATCH_SIZE = 32
    TEST_BATCH_SIZE = 32

    #LEARNING_RATE = 5e-5
    #MAX_EPOCHS = 2
    WEIGHT_DECAY = 0.2
    
    config = {
        "LEARNING_RATE": ray.tune.loguniform(.005, 5e-5),
        #"BATCH_SIZE": tune.choice([32,64,128]),
        "MAX_EPOCHS": ray.tune.choice([2, 5, 8])
    }
    
    metrics = {"loss": "ptl/val_loss", "acc": "ptl/val_accuracy"}
    
    callback = [TuneReportCallback(metrics, on="validation_end")]
        
    SEED = 7
    # DATA_PATH = "/bigtemp/rm5tx/nlp_project/data_cache/"
    DATA_PATH = "/localtmp/rm5tx/nlp_project/data_cache/"
    
    data = ProjData(max_len=128, ratio=2)
    try:
        data.load(DATA_PATH)
        print("Loaded Saved Data")
    except Exception as e: 
        print(e)
        data.setup()
        data.save(DATA_PATH)
    ### Comment out the try block and uncomment below while you're working on the data part or you'll just skip it and use old data.
    # data.setup() 
    # data.save(DATA_PATH)
    
    model = ProjModel(learning_rate = config["LEARNING_RATE"], weight_decay=WEIGHT_DECAY)

    logger = TensorBoardLogger("tb_logs", name="nlp_proj")
   
    def train_tune(config):
        trainer = pl.Trainer(logger=logger, 
                                accelerator='dp', # jupyter can't use ddp, use dp instead
                                # effective batch size is batch_size * num_gpus * num_nodes
                                gpus=1, 
                                gradient_clip_val=1.0, 
                                fast_dev_run=False,
                                progress_bar_refresh_rate=0,
                                callbacks= callbacks
                            )
        trainer.fit(model, data.train_dataloader(batch_size=TRAIN_BATCH_SIZE), 
                        data.val_dataloader(batch_size=VAL_BATCH_SIZE))
    
    from functools import partial
    from ray.tune import CLIReporter
    
    reporter = CLIReporter(
        metric_columns=["loss", "accuracy", "training_iteration"])
    
    analysis = ray.tune.run(
        ray.tune.with_parameters(train_tune, epochs = 10),
        resources_per_trial={"cpu": 2, "gpu": 2},
        config = config,
        num_samples = 5,
        progress_reporter = reporter
    )
    print(analysis.best_config)
    
    model.eval()
    sents = ["random sentence", "pretty flowers", "idiot", "fuck you cunt nigger"]
    for sent in sents:
        x, mask = data.process(sent)
        print(model(x, mask))
                      
    

In [5]:
if __name__ == '__main__':
    main()
    

Loaded Saved Data


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

== Status ==
Memory usage on this node: 42.8/503.1 GiB
Using FIFO scheduling algorithm.
Resources requested: 2.0/40 CPUs, 2.0/4 GPUs, 0.0/317.5 GiB heap, 0.0/140.06 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /u/kem5en/ray_results/train_tune_2021-05-04_22-19-09
Number of trials: 5/5 (4 PENDING, 1 RUNNING)
+------------------------+----------+-------+-----------------+--------------+
| Trial name             | status   | loc   |   LEARNING_RATE |   MAX_EPOCHS |
|------------------------+----------+-------+-----------------+--------------|
| train_tune_477e4_00000 | RUNNING  |       |     9.19498e-05 |            5 |
| train_tune_477e4_00001 | PENDING  |       |     0.000373248 |            8 |
| train_tune_477e4_00002 | PENDING  |       |     0.00360995  |            2 |
| train_tune_477e4_00003 | PENDING  |       |     0.000194505 |            5 |
| train_tune_477e4_00004 | PENDING  |       |     0.000357641 |            5 |
+------------------------+----------+-------+--

2021-05-04 22:19:35,069	WARNING worker.py:1115 -- Failed to unpickle actor class 'ImplicitFunc' for actor ID ebeb91f64e8344cea8b3a1e301000000. Traceback:
Traceback (most recent call last):
  File "/u/kem5en/.conda/envs/ml_proj/lib/python3.6/site-packages/ray/_private/function_manager.py", line 496, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
  File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightning/__init__.py", line 29, in <module>
    from pytorch_lightning.callbacks import Callback  # noqa: E402
  File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightning/callbacks/__init__.py", line 25, in <module>
    from pytorch_lightning.callbacks.swa import StochasticWeightAveraging
  File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightning/callbacks/swa.py", line 26, in <module>
    from pytorch_lightning.trainer.optimizers import _get_default_scheduler_config
  File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightn

Result for train_tune_477e4_00000:
  {}
  
== Status ==
Memory usage on this node: 45.0/503.1 GiB
Using FIFO scheduling algorithm.
Resources requested: 2.0/40 CPUs, 2.0/4 GPUs, 0.0/317.5 GiB heap, 0.0/140.06 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /u/kem5en/ray_results/train_tune_2021-05-04_22-19-09
Number of trials: 5/5 (1 ERROR, 3 PENDING, 1 RUNNING)
+------------------------+----------+-------+-----------------+--------------+
| Trial name             | status   | loc   |   LEARNING_RATE |   MAX_EPOCHS |
|------------------------+----------+-------+-----------------+--------------|
| train_tune_477e4_00001 | RUNNING  |       |     0.000373248 |            8 |
| train_tune_477e4_00002 | PENDING  |       |     0.00360995  |            2 |
| train_tune_477e4_00003 | PENDING  |       |     0.000194505 |            5 |
| train_tune_477e4_00004 | PENDING  |       |     0.000357641 |            5 |
| train_tune_477e4_00000 | ERROR    |       |     9.19498e-05 |           

...

(pid=43821) 2021-05-04 22:19:52,825	ERROR function_manager.py:498 -- Failed to load actor class ImplicitFunc.
(pid=43821) Traceback (most recent call last):
(pid=43821)   File "/u/kem5en/.conda/envs/ml_proj/lib/python3.6/site-packages/ray/_private/function_manager.py", line 496, in _load_actor_class_from_gcs
(pid=43821)     actor_class = pickle.loads(pickled_class)
(pid=43821)   File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightning/__init__.py", line 29, in <module>
(pid=43821)     from pytorch_lightning.callbacks import Callback  # noqa: E402
(pid=43821)   File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightning/callbacks/__init__.py", line 25, in <module>
(pid=43821)     from pytorch_lightning.callbacks.swa import StochasticWeightAveraging
(pid=43821)   File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightning/callbacks/swa.py", line 26, in <module>
(pid=43821)     from pytorch_lightning.trainer.optimizers import _get_default_scheduler_config
(pi

Result for train_tune_477e4_00003:
  {}
  


2021-05-04 22:20:00,056	WARNING worker.py:1115 -- Failed to unpickle actor class 'ImplicitFunc' for actor ID 39e721bd90770eec946c999201000000. Traceback:
Traceback (most recent call last):
  File "/u/kem5en/.conda/envs/ml_proj/lib/python3.6/site-packages/ray/_private/function_manager.py", line 496, in _load_actor_class_from_gcs
    actor_class = pickle.loads(pickled_class)
  File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightning/__init__.py", line 29, in <module>
    from pytorch_lightning.callbacks import Callback  # noqa: E402
  File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightning/callbacks/__init__.py", line 25, in <module>
    from pytorch_lightning.callbacks.swa import StochasticWeightAveraging
  File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightning/callbacks/swa.py", line 26, in <module>
    from pytorch_lightning.trainer.optimizers import _get_default_scheduler_config
  File "/u/kem5en/.local/lib/python3.6/site-packages/pytorch_lightn

Result for train_tune_477e4_00004:
  {}
  
== Status ==
Memory usage on this node: 43.6/503.1 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/40 CPUs, 0/4 GPUs, 0.0/317.5 GiB heap, 0.0/140.06 GiB objects (0.0/1.0 accelerator_type:RTX)
Result logdir: /u/kem5en/ray_results/train_tune_2021-05-04_22-19-09
Number of trials: 5/5 (5 ERROR)
+------------------------+----------+-------+-----------------+--------------+
| Trial name             | status   | loc   |   LEARNING_RATE |   MAX_EPOCHS |
|------------------------+----------+-------+-----------------+--------------|
| train_tune_477e4_00000 | ERROR    |       |     9.19498e-05 |            5 |
| train_tune_477e4_00001 | ERROR    |       |     0.000373248 |            8 |
| train_tune_477e4_00002 | ERROR    |       |     0.00360995  |            2 |
| train_tune_477e4_00003 | ERROR    |       |     0.000194505 |            5 |
| train_tune_477e4_00004 | ERROR    |       |     0.000357641 |            5 |
+--------------------

TuneError: ('Trials did not complete', [train_tune_477e4_00000, train_tune_477e4_00001, train_tune_477e4_00002, train_tune_477e4_00003, train_tune_477e4_00004])